# WQD 7012 Applied Machine Learning Assignment

## Assignment Details

- Title: Optimizing Loan Approval Decisions with Machine Learning Models
- Date: April 8, 2025

## 1. Intial Setup

Import some of the useful libraries and load the data for usage in other section. If you haven't install any of the package, type this in the terminal for installation:

```bash
py -m pip install <package_name>

# if you have a few version of python in your system
py -3.10 pip install <package_name>
py -3.11 pip install <package_name>
py -3.12 pip install <package_name>
py -3.13 pip install <package_name>
```

In [1]:
# import libraries
import pandas as pd
import warnings 
import numpy as np
import os

# turn off warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# load and preview the data
df = pd.read_csv('data/data.csv')
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


## 2. Data Understanding

This section probably will include steps like showing the shape of the data, the summary statistics, counting the unique values, and checking the data types. The goal is to understand the data better before we start the analysis.


In [3]:
# check the shape of the data

## 3. Exploratory Data Analysis

This section will include steps like visualizing the data, checking for missing values, and understanding the distribution of the data. The goal is to understand the data better before we start the analysis.


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

## 4. Data Preprocessing

This section will include steps like handling missing values, encoding categorical variables, and scaling the data. The goal is to prepare the data for modeling.

In [6]:
# shows the missing value
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]

print("Missing values in the dataset:")
print(missing_values)

Missing values in the dataset:
Series([], dtype: int64)


In [8]:
# shows unique value of categorical columns
print("\nUnique values in categorical columns:")
for col in df.select_dtypes(include=['object']).columns:
    unique_values = df[col].unique()
    print(f"{col}: {len(unique_values)} unique values")
    print(unique_values[:5])  # show only the first 5 unique values
    print()


Unique values in categorical columns:
person_gender: 2 unique values
['female' 'male']

person_education: 5 unique values
['Master' 'High School' 'Bachelor' 'Associate' 'Doctorate']

person_home_ownership: 4 unique values
['RENT' 'OWN' 'MORTGAGE' 'OTHER']

loan_intent: 6 unique values
['PERSONAL' 'EDUCATION' 'MEDICAL' 'VENTURE' 'HOMEIMPROVEMENT']

previous_loan_defaults_on_file: 2 unique values
['No' 'Yes']



## 5. Modelling

This section will include steps like splitting the data into training and testing sets, training the model, and evaluating the model. The goal is to build a model that can predict the target variable accurately.

## 6. Evaluation

This section will include steps like evaluating the model using various metrics, such as accuracy, precision, recall, and F1 score. The goal is to understand how well the model performs on the test data.